In [304]:
#CADEC
def tokens_dict(filepath):
    f = open(filepath, 'r', encoding='utf-8')
    dict_txt = {}
    word = []
    file_name = ""
    start = 0
    end = 0
    for line in f:
        
        if len(line) != 1:
            line_spl = line.split()
            file_name = line_spl[1]
            if line_spl[1] not in dict_txt:
                count = 0
                word = []
                start = 0
                dict_txt[file_name] = []

            word.append(line_spl[0])
        else:
            end = int(line_spl[-1])
            dict_txt[file_name].append((start, end))
            start = end
            count += 1
            word = []
    f.close()
    return dict_txt

filepath = 'acl/cadec/tokens'
dict_txt = tokens_dict(filepath)

            

In [358]:

import re
import pandas as pd
import json
import os
# import numpy as np
from operator import itemgetter
ans = pd.read_csv('acl/cadec/ann', names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
ans_tmp = ans.groupby(['file_id'])
# ans.fillna("NA", inplace=True)
ans.head()
model="w2ner"
data_format = "train"        #dev, train/test
file_format = "cadec"       #share13/share14/cadec
ans_data_path = "train"      #dev, train /test

#share13
to_enhance_count = "enhance_model_cadec_our_split_1_new_inner"
split_type="our"
#enhance_share13_new_origin_1/enhance_share13_share13_origin_1
model="w2ner"
if "cadec" in file_format:
    data_format = data_format        #dev, train/test
    file_format = 'cadec'      #share13/share14/cadec
    ans_data_path = data_format      #dev, train /test
    answer_file='acl/cadec/ann'
    folder_path = 'acl/'+file_format+'/text'
    # path = "acl/data/cadec/new_"+file_format+"/"+file_format+"_"+data_format+".json"
    path = "acl/"+file_format+"/"+file_format+"_word_list.json"
    pattern = r'[\w]+|[^\s\w]'
    pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
else:
    data_format = data_format        #dev, train/test
    if "share13" in file_format:
        file_format = 'share13'       #share13/share14/cadec
    else:
        file_format = 'share14'      #share13/share14/cadec

    if ans_data_path=="test":
        ans_data_path=ans_data_path
    else:
        ans_data_path="train"
    answer_file='acl/'+file_format+'/'+ans_data_path+'.ann'
    folder_path = 'acl/'+file_format+'/'+ans_data_path+'/text'
    if "our" in split_type:
        # our pattern 
        pattern = r'[\w]+|[^\s\w]'
        pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
        path = "acl/"+file_format+"/"+file_format+"_word_list.json"
    if "dai" in split_type:
        # new pattern - dai
        contractions = set(["n't", "'s", "'ve", "'re", "'ll", "'d", "'m"])
        contractions |= set([x.replace("'", "’") for x in contractions])

        contractions_pattern_1 = "\w+(?="+")|\w+(?=".join(list(contractions))+")"
        contractions_pattern_2 = "|".join(list(contractions))

        pattern = "\w+(?=n't)|~\w+|"+contractions_pattern_1+"|\w+#\w*|\w+=\w*|\w+‘\w*|\w+“\w*|"+contractions_pattern_2+"|"+r"""[\w\d]+[*]+|[|]+|\w+"\w+|\w+'\w+|[\w]+|[^\s\w]"""
        pattern_s = "\w+(?=n't)|~\w+|"+contractions_pattern_1+"|\w+#\w*|\w+=\w*|\w+‘\w*|\w+“\w*|"+contractions_pattern_2+"|"+r"""[\w\d]+[*]+|[|]+|\w+"\w+|\w+'\w+|[\w]+|[^\s\w]"""+"|\n|[\s]+"
        path = "D:/hw/W2NER-final/acl/data/cadec/new_"+file_format+"/"+file_format+"_"+data_format+"_new.json"
    


ann_own_path = "ann"  #ann_vik / ann
# #cadec
# answer_file='D:/acl/data/cadec/adr/ann'
#share13/14
ans = pd.read_csv(answer_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
ans_tmp = ans.groupby(['file_id'])
sentence_length = []
enhance_sentence_length = []
# 設定輸出 JSON 檔案的路徑
output_path = "D:/hw/W2NER-final/data/cadec_entity_bound/"+data_format+"_entity.json"

main_block_size =0
ES_action_state  = 0
NES_action_state = 0
is_write =       1      # 1寫入        / 0 不寫
get_all_entity = 1     # 1取得全部實體 / 0 則否
total_sentence = 0 
add_train_random = 0            #train random 0/1 沒有/有
contain_merge_conflict_span = 0 # 0 衝突區間個體/ 1 衝突區間全部

'''
action_state
{
    0: not work
    1: work
}
'''
support_strategy = 2
'''
support_strategy
{
    0: look forward
    1: look backward
    2: look both side
}
'''
look_forward_step = 4
look_backward_step= 4

if main_block_size==0:
    main_block_mode = "dynamic_size"
else:
    main_block_mode = str(main_block_size)
# to_enhance_count
if support_strategy==0 and (ES_action_state or NES_action_state):
    to_enhance_count += "_look_forward"
    domain_t = main_block_mode+"-"+"F"+str(look_forward_step)+"-"+"B0"
elif support_strategy==1 and (ES_action_state or NES_action_state):
    to_enhance_count += "_look_backward"
    domain_t = main_block_mode+"-"+"F0"+"-"+"B"+str(look_backward_step)
elif support_strategy==2 and (ES_action_state or NES_action_state):
    to_enhance_count += "_look_both_side"
    domain_t = main_block_mode+"-"+"F"+str(look_forward_step)+"-"+"B"+str(look_backward_step)
else:
    to_enhance_count += ""

if ES_action_state + NES_action_state==2:
    domain_s = "_ALL"+"-"+domain_t
elif ES_action_state==1:
    domain_s = "_ES"+"-"+domain_t
elif NES_action_state==1:
    domain_s = "_NES"+"-"+domain_t
else:
    domain_s = "_None"

merge_intervals_limit = 5 # char 數為單位
entity_intervallimit = 20       # 設35會錯

total_entity_number = 0
avg_sentence_len = 0
total_sentence = 0
total_entity = 0
count = 0
single_entity = 0

with open(path)as f_ori:
    dic_f_wordlist = json.load(f_ori)

def index_label(test_list):
    # test_list = [[1940, 1952], [1960, 1970], ['MCA aneurysm', 'moving all']]
    pre_text = test_list[-1]
    newList = []
    textList = []
    start = 0
    # pattern = r'[\w]+|\n|[^\s\w]'
    for i in range(len(pre_text)):
        start =  test_list[i][0]
        p_t_s = re.findall(pattern_s,pre_text[i])
        # p_t = re.findall(pattern,pre_text[i])
        for it,t in enumerate(p_t_s):        
            end = start + len(t)
            
            if t != ' ':
                textList.append(t)
                
                newList.append([start, end])
            start = end
    newList.append(textList)
    return newList

def search_entity_index(sentence, ran, d):
    
    list_ner = []
    # print(sentence)
    for i in range(len(d[-2])):
        
        entity_start = d[i][0] - ran[0]
        entity_end = d[i][1] - ran[0]
        # print(entity_start, entity_end)
        if i == 0:
            # re.findall(pattern,sentence[0:entity_start])
            entity_ner = len(re.findall(pattern,sentence[0:entity_start]))
            # print(entity_ner)
            list_ner.append(entity_ner)
            
        else:
            
            entity_ner = entity_ner + len(re.findall(pattern,sentence[entity_record:entity_start])) + 1
            if entity_ner < len(re.findall(pattern,sentence)):
                list_ner.append(entity_ner)
        entity_record = entity_end
        
    # print(list_ner)
    return list_ner

def merge_span(ner_start_end_dict):
    new_item_s_e = []
    for k, vv in ner_start_end_dict.items():
        new_item_s_e.append(vv)
    new_item_s_e = sorted(new_item_s_e,key=itemgetter(0))
    mergedData = []
    start, end = new_item_s_e[0]
    for pair in new_item_s_e[1:]:
        if pair[0] <= end:
            end = max(end, pair[1])
        else:
            mergedData.append([start, end])
            start, end = pair

    mergedData.append([start, end])
    return mergedData

def sentence_ner(diease, sentence_dict):
    global single_entity
    ans = {}
    ans_single = {}
    # print(sentence_dict)
    for d in diease:
        for k, v in sentence_dict.items():
            sentence = v
            ran = k
            if ran[0]<=d[0][0] and d[-3][-1]<=ran[1]:

                list_ner = search_entity_index(sentence, ran, d)        
                # print(list_ner)
                if ran not in ans:     
                    ans[ran] = []
                    ans[ran].append({"index":list_ner, "type":d[-1]})
                else:
                    ans[ran].append({"index":list_ner, "type":d[-1]})    
                single_entity+=1
                break
    # print(ans)
    for k, v in ans.items():
        if len(v) >1:
            merge_dict = {}
            merge_list = []
            for i in v:
                if i["index"][0] not in merge_dict:
                    merge_dict[i["index"][0]] = [i["index"][0], i["index"][-1]]
                else:
                    nl = max(merge_dict[i["index"][0]][-1], i["index"][-1])
                    merge_dict[i["index"][0]] = [i["index"][0], nl]
           
            mergedData = merge_span(merge_dict)
            merge_list = []
            for i in mergedData:
                merge_list.append({'index': [m for m in range(int(i[0]), int(i[-1])+1)], 'type': d[-1]})
            ans[k] = merge_list
    # print(ans)
    return ans
def ans_list(ans, sentence_dict, filename):
    list_dict = []
    count = 0
    start=0
    end = 0
    for k, v in sentence_dict.items():
        # json_dict = {"sentence":[], "ner":[], "filename": filename, "word_start_end":k}
        word_list = re.findall(pattern_s, v)
        # print(word_list)
        # word_index_list = word_index(word_list)
        word_index_list = []
        for word in word_list:
            end = start+len(word)
            if word.strip()!="":
                word_index_list.append([start, end])
            start=end
        json_dict = {"sentence":[], "ner":[], "filename": filename, "word_start_end":word_index_list}
   
        json_dict["sentence"] = re.findall(pattern, v)
        if k in ans:
            json_dict["ner"] = ans[k]
            count += len(ans[k])
        if json_dict["sentence"] != []:
            list_dict.append(json_dict)
    return list_dict, count



with open(output_path, 'w', encoding='utf-8') as json_file:
    all_list = []
    nn = 0
    f_name = open("answer/"+file_format+"/id/"+data_format+".id","r").read().split("\n")
    # for filename in os.listdir(folder_path):
    for filename in f_name:
        # file_path = folder_path+"/"+filename
        # filename = 'LIPITOR.553'
        if file_format=="cadec":
            file_path = folder_path+"/"+filename +".txt"
        else:
            file_path = folder_path+"/"+filename
        with open(file_path, encoding='utf-8', errors='ignore') as f:
            doc_file = f.read()
            
                    
            if doc_file=="":
                continue
            try:
                true_answers = ans_tmp.get_group(filename)
            except:
                # doc_file
                # print(file_name)
                sentence_dict = {}
                start=0
                end = 0
                for s_e in dict_txt[filename]:
                    # sentence_dict[s_e] = doc_file[s_e[0]:s_e[1]]
                    sentence = doc_file[s_e[0]:s_e[1]]
                    word_list = re.findall(pattern_s, sentence)
                    # word_index_list = word_index(word_list)
                    word_index_list = []
                    for word in word_list:
                        end = start+len(word)
                        if word.strip()!="":
                            word_index_list.append([start, end])
                        start=end
                    # word_index_list
                    if re.findall(pattern, sentence) != []:
                        all_list.append({"sentence":re.findall(pattern, sentence), "ner":[], "filename": filename, "word_start_end":word_index_list})                    
                continue
                # not_in_answer_content = re.findall(pattern,doc_file)
                # for e_i in range(0,len(not_in_answer_content),block_size):
                #     all_list.append({"sentence":not_in_answer_content[e_i:e_i+block_size], "ner":[]})
            target_start_end = true_answers[['start_end']]
            target_entity = true_answers[['entity']]
            target_other = true_answers[['other']]
            #---------------------------------------------------
            index_to_int = []
            target_entity_list = target_entity.values.tolist()
            target_label = true_answers[['label']].values.tolist()
            for s_e_index, s_e in enumerate(target_start_end.values.tolist()):
                s_e = [int(span) for span in s_e[0].split(',')]
                # index_to_int.append([s_e, target_entity_list[s_e_index][0]])
                index_to_int.append([s_e, target_entity_list[s_e_index][0],target_label[s_e_index][0]])
            index_to_int = sorted(index_to_int,key=itemgetter(0))
            
            items_list = []
            item_s_e = {}
            for item in index_to_int:
                items = []
                # print(item)
                if len(item[0])>2:
                    start = 0
                    item_str = []
                    item_index = []
                    itm = item[1]
                    for index in range(0, len(item[0]), 2):     
                        end = start + (item[0][index+1] - item[0][index])
                        item_str.append(itm[start:end])
                        itm = itm[end:].strip()
                        item_index.append([item[0][index], item[0][index+1]])
                    items = item_index+[item_str]
                else:
                    items = [item[0], [item[1]]]
                
                
                newList = index_label(items)+[item[2]]
                
                # print(newList)
                if newList[0][0] not in item_s_e:
                    item_s_e[newList[0][0]] = [newList[0][0], newList[-2][-1]]
                else:
                    nl = max(item_s_e[newList[0][0]][-1] ,newList[-2][-1])
                    item_s_e[newList[0][0]] = [newList[0][0], nl]
                # print(newList)
                items_list.append(newList)
            # print(items_list)
            sentence_dict = {}
            for s_e in dict_txt[filename]:
                sentence_dict[s_e] = doc_file[s_e[0]:s_e[1]]
            # print(items_list)
            ans = sentence_ner(items_list, sentence_dict)
            
            list_dict, ner_count = ans_list(ans, sentence_dict, filename)
            # print(ner_count)
            nn+=ner_count
            # break
            # assert ner_count==len(target_entity)
            all_list.extend(list_dict)
    json.dump(all_list, json_file, ensure_ascii=False)

In [415]:
from operator import itemgetter
def merge_span(ner_start_end_dict):
    new_item_s_e = []
    for v in ner_start_end_dict:
        new_item_s_e.append(v)
    new_item_s_e = sorted(new_item_s_e,key=itemgetter(0))
    #[12,20],[13,19],[14,21] => [12,21]
    mergedData = []
    start, end = new_item_s_e[0]
    for pair in new_item_s_e[1:]:
        if pair[0] <= end:
            end = max(end, pair[1])
        else:
            mergedData.append([start, end])
            start, end = pair

    mergedData.append([start, end])
    # mergedDataArray= np.array(mergedData)
    return mergedData
answer_start_end_path = "D:/hw/W2NER-final/answer/cadec/cadec_"+ "train" +"_answer.json"
entity_bound_set= {}
c=0
with open(answer_start_end_path, 'r', encoding='utf-8') as json_file:
    answer_start_end = json.load(json_file)
    for file_name,start_end in answer_start_end.items():
        
        start_end = [[int(s_e.split(',')[0]),int(s_e.split(',')[-1])] for s_e in start_end]
        
        start_end = merge_span(start_end)
        
        entity_bound_set[file_name] = set([",".join(map(str, s_e)) for s_e in start_end])
        c+=len(entity_bound_set[file_name])
        # if file_name == "LIPITOR.269":
        #     print(start_end)
        #     print(file_name, len(entity_bound_set[file_name]))
        #     print(entity_bound_set[file_name])

In [361]:
entity_bound_set

{'LIPITOR.553': {'135,139', '158,173', '3,25', '92,131'},
 'DICLOFENAC-SODIUM.3': {'129,137',
  '142,156',
  '171,187',
  '194,208',
  '228,236',
  '31,49',
  '353,361',
  '526,542',
  '553,561',
  '569,585',
  '60,74',
  '7,23',
  '94,105'},
 'LIPITOR.355': {'0,25', '148,164', '188,192', '352,356', '47,80'},
 'LIPITOR.320': {'0,28', '172,199', '33,50', '71,102'},
 'LIPITOR.840': {'0,11',
  '105,147',
  '13,57',
  '149,182',
  '190,210',
  '215,232',
  '307,318',
  '379,413',
  '420,442',
  '487,511',
  '573,578',
  '59,103'},
 'LIPITOR.15': {'119,137', '183,185'},
 'LIPITOR.804': {'0,9', '11,20', '22,32'},
 'LIPITOR.777': {'103,124', '162,197', '259,280', '286,293'},
 'VOLTAREN.1': {'0,9', '14,27'},
 'LIPITOR.474': {'119,126', '15,25', '58,66'},
 'ARTHROTEC.88': {'3,18', '31,43', '92,100'},
 'LIPITOR.468': {'115,129',
  '134,141',
  '143,185',
  '204,213',
  '380,401',
  '429,436',
  '441,447',
  '452,457',
  '71,85'},
 'ARTHROTEC.104': {'20,32', '34,41', '5,13'},
 'LIPITOR.558': {'57

In [216]:
entity_bound_set['LIPITOR.553'],c

({'135,139', '158,173', '3,25', '92,131'}, 918)

In [270]:
def merge_span(ner_start_end_dict):
    new_item_s_e = []
    for k, v in ner_start_end_dict.items():
        new_item_s_e.append(v)
    new_item_s_e = sorted(new_item_s_e,key=itemgetter(0))
    mergedData = []
    start, end = new_item_s_e[0]
    for pair in new_item_s_e[1:]:
        if pair[0] <= end:
            end = max(end, pair[1])
        else:
            mergedData.append([start, end])
            start, end = pair

    mergedData.append([start, end])
    return mergedData

ans = {(0, 85):[{'index': [1, 2, 3], 'type': 'diease'}], 
       (85, 211): [{'index': [2, 3, 4, 5, 6], 'type': 'diease'}, 
                   {'index': [2, 3, 4, 5, 8], 'type': 'diease'}, 
                   {'index': [2, 3, 4, 5, 10], 'type': 'diease'}, 
                   {'index': [13], 'type': 'diease'}, 
                   {'index': [18, 19], 'type': 'diease'}]}
mergedData = merge_span(ans)


[{'index': [1, 2, 3], 'type': 'diease'}]
[{'index': [2, 3, 4, 5, 6], 'type': 'diease'}, {'index': [2, 3, 4, 5, 8], 'type': 'diease'}, {'index': [2, 3, 4, 5, 10], 'type': 'diease'}, {'index': [13], 'type': 'diease'}, {'index': [18, 19], 'type': 'diease'}]


In [350]:
# ans = {(0, 85):[{'index': [2, 3, 4, 5], 'type': 'ADR'}, {'index': [4, 5], 'type': 'ADR'}], 
#        (85, 211): [{'index': [2, 3, 4, 5, 6], 'type': 'diease'}, 
#                    {'index': [2, 3, 4, 5, 8], 'type': 'diease'}, 
#                    {'index': [2, 3, 4, 5, 10], 'type': 'diease'}, 
#                    {'index': [13], 'type': 'diease'}, 
#                    {'index': [18, 19], 'type': 'diease'}]}
ans = {(0, 77): [{'index': [1, 2], 'type': 'ADR'}, {'index': [4, 5], 'type': 'ADR'}, 
                 {'index': [7, 8], 'type': 'ADR'}, {'index': [7, 10, 11, 12], 'type': 'ADR'}], 
       (77, 160): [{'index': [10, 11], 'type': 'ADR'}, {'index': [10, 13], 'type': 'ADR'}]}

def merge_span(ner_start_end_dict):
    new_item_s_e = []
    for k, v in ner_start_end_dict.items():
        new_item_s_e.append(v)
    # print(new_item_s_e)
    new_item_s_e = sorted(new_item_s_e,key=itemgetter(0))
    # print(new_item_s_e)
    mergedData = []
    start, end = new_item_s_e[0]
    for pair in new_item_s_e[1:]:
        if pair[0] <= end:
            end = max(end, pair[1])
        else:
            mergedData.append([start, end])
            start, end = pair

    mergedData.append([start, end])
    return mergedData

for k, v in ans.items():
    if len(v) >1:
        merge_dict = {}
        merge_list = []
        for i in v:
            if i["index"][0] not in merge_dict:
                merge_dict[i["index"][0]] = [i["index"][0], i["index"][-1]]
            else:
                nl = max(merge_dict[i["index"][0]][-1], i["index"][-1])
                merge_dict[i["index"][0]] = [i["index"][0], nl]
        mergedData = merge_span(merge_dict)
        # print(mergedData)
        merge_list = []
        for i in mergedData:
            # print(i)
            merge_list.append({'index': [m for m in range(int(i[0]), int(i[-1])+1)], 'type': 'diease'})

        ans[k] = merge_list
            

In [351]:
mergedData

[[10, 13]]

In [80]:
merge_list

[{'index': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'type': 'diease'},
 {'index': [13], 'type': 'diease'},
 {'index': [18, 19], 'type': 'diease'}]

In [52]:
merge_dict

{2: [2, 10], 13: [13, 13], 18: [18, 19]}

In [44]:
for i in merge_dict.values():
    
    print([m for m in range(i[0], i[-1]+1)])

[2, 3, 4, 5, 6, 7, 8, 9, 10]
[13]
[18, 19]


In [40]:
merge_dict

{2: [2, 10], 13: [13, 13], 18: [18, 19]}

In [385]:
from operator import itemgetter
def merge_span(ner_start_end_dict):
    new_item_s_e = []
    for v in ner_start_end_dict:
        new_item_s_e.append(v)
    new_item_s_e = sorted(new_item_s_e,key=itemgetter(0))
    #[12,20],[13,19],[14,21] => [12,21]
    mergedData = []
    start, end = new_item_s_e[0]
    for pair in new_item_s_e[1:]:
        if pair[0] <= end:
            end = max(end, pair[1])
        else:
            mergedData.append([start, end])
            start, end = pair

    mergedData.append([start, end])
    # mergedDataArray= np.array(mergedData)
    return mergedData
answer_start_end_path = r"data\cadec_entity_bound\train_entity.json"
entity_bound_set= {}

with open(answer_start_end_path, 'r', encoding='utf-8') as json_file:
    ansstart_end = json.load(json_file)
    count = 0
    ff = ansstart_end[0]["filename"]
    print(a["filename"])
    for a in ansstart_end:
        if ff == a["filename"]:
            count+=len(a["ner"])
            # print(a["ner"])
        else:
            if count!=0:
                print(ff, count)
            ff = a["filename"]
            count = len(a["ner"])

    
     

LIPITOR.635
LIPITOR.774 3
LIPITOR.896 7
VOLTAREN.33 2
LIPITOR.845 2
LIPITOR.965 12
LIPITOR.878 8
LIPITOR.473 6
LIPITOR.691 6
LIPITOR.477 3
LIPITOR.226 6
LIPITOR.670 2
LIPITOR.614 4
LIPITOR.233 2
LIPITOR.652 15
LIPITOR.705 6
ARTHROTEC.42 22
LIPITOR.563 12
LIPITOR.884 3
LIPITOR.387 5
LIPITOR.574 6
DICLOFENAC-SODIUM.5 2
LIPITOR.208 4
LIPITOR.798 6
LIPITOR.731 2
LIPITOR.478 5
ARTHROTEC.119 3
LIPITOR.872 8
LIPITOR.649 3
LIPITOR.440 2
LIPITOR.63 6
LIPITOR.771 5
LIPITOR.458 5
LIPITOR.981 8
CATAFLAM.3 4
LIPITOR.966 9
LIPITOR.389 14
LIPITOR.719 1
LIPITOR.863 3
LIPITOR.592 7
LIPITOR.332 2
LIPITOR.386 8
LIPITOR.304 11
LIPITOR.848 1
LIPITOR.599 1
LIPITOR.631 9
ARTHROTEC.62 2
LIPITOR.138 5
LIPITOR.227 7
LIPITOR.139 2
LIPITOR.112 2
LIPITOR.270 5
LIPITOR.210 3
LIPITOR.388 13
LIPITOR.596 3
LIPITOR.155 8
LIPITOR.339 9
LIPITOR.766 9
ARTHROTEC.72 1
LIPITOR.542 1
LIPITOR.132 2
LIPITOR.929 1
LIPITOR.488 14
LIPITOR.946 3
LIPITOR.151 4
LIPITOR.117 3
LIPITOR.395 7
ARTHROTEC.71 4
LIPITOR.531 3
LIPITOR.289 6
LI

In [413]:
# def merge_span(ner_start_end_dict):
ner_start_end_dict = [[0, 11], [13, 21], [23, 30], [32, 43], [48, 57], [66, 74], [79, 87],
                       [103, 106], [108, 116], [117, 128], [129, 140], [141, 153], [153, 187],
                         [223, 228], [254, 320], [322, 345], [347, 359], [362, 380], [381, 393],
                           [394, 404], [405, 424], [425, 449], [425, 459], [460, 477], [478, 503],
                             [540, 550], [563, 570], [578, 587], [856, 888],
                       [890, 911], [901, 911], [1305, 1316], [1321, 1329], [1330, 1345], [1346, 1364]]
new_item_s_e = []
for v in ner_start_end_dict:
    new_item_s_e.append(v)
new_item_s_e = sorted(new_item_s_e,key=itemgetter(0))
# #[12,20],[13,19],[14,21] => [12,21]
mergedData = []
start, end = new_item_s_e[0]
for pair in new_item_s_e[1:]:
    # print(pair)
    if pair[0] < end:
        # print(pair[0])
        end = max(end, pair[1])
    else:
        mergedData.append([start, end])
        start, end = pair

mergedData.append([start, end])

In [414]:
mergedData

[[0, 11],
 [13, 21],
 [23, 30],
 [32, 43],
 [48, 57],
 [66, 74],
 [79, 87],
 [103, 106],
 [108, 116],
 [117, 128],
 [129, 140],
 [141, 153],
 [153, 187],
 [223, 228],
 [254, 320],
 [322, 345],
 [347, 359],
 [362, 380],
 [381, 393],
 [394, 404],
 [405, 424],
 [425, 459],
 [460, 477],
 [478, 503],
 [540, 550],
 [563, 570],
 [578, 587],
 [856, 888],
 [890, 911],
 [1305, 1316],
 [1321, 1329],
 [1330, 1345],
 [1346, 1364]]

In [404]:
len(new_item_s_e)

35

In [384]:
a

{'sentence': ['After',
  'I',
  'quit',
  'taking',
  'Lipitor',
  'for',
  '3',
  'days',
  'the',
  'pain',
  'was',
  'almost',
  'gone',
  '.'],
 'ner': [{'index': [9], 'type': 'ADR'}],
 'filename': 'LIPITOR.635',
 'word_start_end': [[68, 73],
  [74, 75],
  [76, 80],
  [81, 87],
  [88, 95],
  [96, 99],
  [100, 101],
  [102, 106],
  [107, 110],
  [111, 115],
  [116, 119],
  [120, 126],
  [127, 131],
  [131, 132]]}